In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.4 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,335 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd6

In [7]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

In [8]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [9]:
from google.colab import files

uploaded = files.upload()

Saving NFL_BOTH.csv to NFL_BOTH.csv


In [10]:
import pandas as pd
import io
  
df = pd.read_csv('NFL_BOTH.csv', header= 0,
                        encoding= 'unicode_escape')
df.head()

,ï»¿Year,ACC,Big_Ten,Big_12,Pac_12,SEC,Other,Height_IN,Weight_LBS,Wonderlic,40_Yard,Bench_Press,Vert_Leap_IN,Broad_Jump_IN,Shuttle,3Cone,Started
0,2020,0,0,0,1,0,0,78.13,224,NaN,5.07,NaN,28.5,NaN,4.60,7.85,Yes
1,2019,0,0,0,0,1,0,76.63,226,NaN,4.64,NaN,29.5,115.0,4.56,7.42,No
2,2019,0,0,0,1,0,0,73.50,202,NaN,4.93,NaN,27.0,NaN,NaN,NaN,No
3,2019,0,0,0,0,0,1,75.25,233,NaN,5.22,NaN,NaN,NaN,4.68,NaN,No
4,2019,0,0,0,0,0,1,72.75,206,NaN,4.76,NaN,35.0,110.0,4.32,NaN,No


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ï»¿Year        54 non-null     int64  
 1   ACC            54 non-null     int64  
 2   Big_Ten        54 non-null     int64  
 3   Big_12         54 non-null     int64  
 4   Pac_12         54 non-null     int64  
 5   SEC            54 non-null     int64  
 6   Other          54 non-null     int64  
 7   Height_IN      54 non-null     float64
 8   Weight_LBS     54 non-null     int64  
 9   Wonderlic      10 non-null     float64
 10  40_Yard        53 non-null     float64
 11  Bench_Press    6 non-null      float64
 12  Vert_Leap_IN   47 non-null     float64
 13  Broad_Jump_IN  44 non-null     float64
 14  Shuttle        41 non-null     float64
 15  3Cone          38 non-null     float64
 16  Started        54 non-null     object 
dtypes: float64(8), int64(8), object(1)
memory usage: 7.3+ KB

In [12]:
df.drop(columns=["Wonderlic","Bench_Press"],axis=1, inplace=True)

In [15]:
df.drop(columns=["ï»¿Year", "ACC", "Big_Ten", "Big_12", "SEC", "Other"],axis=1,inplace=True )

In [17]:
df.drop(columns=["Pac_12"],axis=1, inplace=True)

In [18]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 1 to 52
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Height_IN      32 non-null     float64
 1   Weight_LBS     32 non-null     int64  
 2   40_Yard        32 non-null     float64
 3   Vert_Leap_IN   32 non-null     float64
 4   Broad_Jump_IN  32 non-null     float64
 5   Shuttle        32 non-null     float64
 6   3Cone          32 non-null     float64
 7   Started        32 non-null     object 
dtypes: float64(6), int64(1), object(1)
memory usage: 2.2+ KB


In [19]:
X = pd.get_dummies(df)
X

,Height_IN,Weight_LBS,40_Yard,Vert_Leap_IN,Broad_Jump_IN,Shuttle,3Cone,Started_No,Started_Yes
1,76.63,226,4.64,29.5,115.0,4.56,7.42,1,0
14,74.38,230,4.91,30.0,117.0,4.31,7.75,1,0
15,78.13,234,4.93,29.0,105.0,4.48,7.42,0,1
16,78.25,237,5.12,27.0,101.0,4.60,7.56,1,0
19,73.75,218,4.89,32.5,112.0,4.32,6.99,1,0
20,74.50,213,4.84,34.0,121.0,4.35,6.97,0,1
21,71.75,191,4.76,31.5,111.0,4.35,7.06,1,0
22,72.38,211,4.76,35.0,123.0,4.21,6.96,1,0
23,73.50,198,4.84,31.0,122.0,4.16,6.80,1,0
24,76.50,248,4.99,31.5,113.0,4.51,7.30,1,0


In [20]:
X.drop(columns=["Started_No"],axis=1, inplace=True)

In [21]:
X.head()

,Height_IN,Weight_LBS,40_Yard,Vert_Leap_IN,Broad_Jump_IN,Shuttle,3Cone,Started_Yes
1,76.63,226,4.64,29.5,115.0,4.56,7.42,0
14,74.38,230,4.91,30.0,117.0,4.31,7.75,0
15,78.13,234,4.93,29.0,105.0,4.48,7.42,1
16,78.25,237,5.12,27.0,101.0,4.60,7.56,0
19,73.75,218,4.89,32.5,112.0,4.32,6.99,0


In [22]:
y=X["Started_Yes"]
X.drop(columns=["Started_Yes"],axis=1, inplace=True)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7916666666666666
Testing Data Score: 0.875


In [28]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [30]:
def getTestData(Height_IN,	Weight_LBS,	40_Yard,	Vert_Leap_IN,	Broad_Jump_IN,	Shuttle,	3Cone):
  testData = [Height_IN,	Weight_LBS,	40_Yard,	Vert_Leap_IN,	Broad_Jump_IN,	Shuttle,	3Cone]
  return testData

SyntaxError: ignored

In [31]:
new_data=[75.13,	229.0,		4.69	,	35.0,	125.0,	4.51,	7.33]

In [32]:
import numpy as np
testData=np.reshape(new_data,(1,-1))

In [33]:
predictions = classifier.predict(testData)
print("Players are either 0 (not a starter) or 1 (stared)")
print(f"The player was classified as: {predictions}")

Players are either 0 (not a starter) or 1 (stared)
The player was classified as: [0]
